In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torchvision
from sklearn.metrics import confusion_matrix  
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import VisionDataset
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import os.path
import sys
from torch.backends import cudnn
import numpy as np
import matplotlib.pyplot as plt

from datasets import Cityscapes
from models import *
from utils import *

In [ ]:
#DEVICE = 'cuda' # 'cuda' or 'cpu'
DEVICE = 'cuda'
NUM_CLASSES = 19 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 16     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1*5e-3           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default -5

NUM_EPOCHS =200      # 20/30 Total number of training epochs (iterations over dataset)
STEP_SIZE = 60       #20 How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.05
POWER = 0.9        
LOG_FREQUENCY = 20

In [ ]:
#transformations
size = (256, 512) #512,1024
scales=(0.25, 2.)
cropsize=(512, 1024)
eval_crop=(1024, 1024)
eval_scales=(0.5, 0.75, 1.0, 1.25, 1.5, 1.75)
test_transformations = [RandomResizedCrop(eval_scales, eval_crop)]
train_transformations = [RandomResizedCrop(scales,cropsize),RandomHorizontalFlip(),ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4)]
test_transformations = Compose(test_transformations)
train_transformations = Compose(train_transformations)


In [ ]:
train_dataset_B = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=train_transformations, split='train')
print(len(train_dataset_B))
test_dataset_B = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='val')
print(len(test_dataset_B))
train_dataset_A = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=train_transformations, split='trainA')
print(len(train_dataset_A))
test_dataset_A = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='valA')
print(len(test_dataset_A))
img,label = train_dataset_B.__getitem__(0)


500
250
708
42
tensor([[[-0.1373, -0.2784, -0.3490,  ..., -0.8980, -0.8902, -0.8824],
         [-0.2471, -0.3490, -0.4118,  ..., -0.8980, -0.8980, -0.8902],
         [-0.4745, -0.5294, -0.5294,  ..., -0.9059, -0.9059, -0.8980],
         ...,
         [-0.2157, -0.2235, -0.2235,  ..., -0.4745, -0.4824, -0.4980],
         [-0.2235, -0.2314, -0.2314,  ..., -0.4745, -0.4902, -0.4824],
         [-0.2314, -0.2471, -0.2471,  ..., -0.4745, -0.4824, -0.4824]],

        [[-0.0196, -0.1059, -0.2000,  ..., -0.8667, -0.8588, -0.8431],
         [-0.0980, -0.1686, -0.2471,  ..., -0.8745, -0.8588, -0.8510],
         [-0.2549, -0.3020, -0.3647,  ..., -0.8745, -0.8667, -0.8667],
         ...,
         [-0.1373, -0.1294, -0.1373,  ..., -0.4118, -0.4118, -0.4353],
         [-0.1373, -0.1373, -0.1373,  ..., -0.4118, -0.4275, -0.4118],
         [-0.1451, -0.1451, -0.1529,  ..., -0.4118, -0.4118, -0.4118]],

        [[-0.1843, -0.2078, -0.2392,  ..., -0.8980, -0.8902, -0.8824],
         [-0.2863, -0.3255, -0

In [ ]:
train_dataloader_B = DataLoader(train_dataset_B, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
test_dataloader_B = DataLoader(test_dataset_B, batch_size=1, shuffle=False, num_workers=2)
train_dataloader_A = DataLoader(train_dataset_A, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
test_dataloader_A = DataLoader(test_dataset_A, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
bisenet_model = BiSeNetV2(n_classes=NUM_CLASSES,output_aux=True,pretrained=True) #meglio toglierla perchè genera bias
bisenet_model.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=255,reduction='none') # da consegna ignore_index=255
parameters_to_optimize = bisenet_model.parameters() 
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
def plotLoss(train,label):
  epochs = np.array([a for a in range(NUM_EPOCHS)]).reshape(NUM_EPOCHS,1)
  plt.figure()
  #plt.plot(epochs,val,label='val_loss')
  plt.ylim(0,max(train)+1)
  plt.plot(epochs,train,label=label)
  plt.legend()
  plt.show()

def plotmIoU(train,label):
  epochs = np.array([a for a in range(NUM_EPOCHS)]).reshape(NUM_EPOCHS,1)
  plt.figure()
  #plt.plot(epochs,val,label='val_loss')
  plt.ylim(0,max(train)+0.1)
  plt.plot(epochs,train,label=label)
  plt.legend()
  plt.show()

In [ ]:
def _fast_hist(n_classes, label_true, label_pred):
      mask = (label_true >= 0) & (label_true < n_classes)
      hist = np.bincount(
          n_classes * label_true[mask].astype(int) + label_pred[mask],
          minlength=n_classes ** 2,
      ).reshape(n_classes, n_classes)
      return hist


def compute_mIoU(y_true,y_pred):
  
  y_pred = y_pred.cpu().numpy().flatten()
  y_true = y_true.cpu().numpy().flatten()
  hist = _fast_hist(19,y_true,y_pred)
  gt_sum = hist.sum(axis=1)
  mask = (gt_sum != 0)
  diag = np.diag(hist)
  iu = diag / (gt_sum + hist.sum(axis=0) - diag)
  mean_iu = np.mean(iu[mask])
  return mean_iu

In [ ]:

checkpoint = torch.load('/content/drive/MyDrive/magna/step2/long_esp_dataset_B_5e-3/180checkpoint.pt')
bisenet_model.load_state_dict(checkpoint['model_state_dict'])
optimizer = optim.SGD(bisenet_model.parameters(), lr=1*1e-4, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
bisenet_model.to(DEVICE)
check_epoch = checkpoint['epoch']
check_loss = checkpoint['loss']
loss_for_epochs = checkpoint['loss_for_epochs']
mIoU_for_epochs = checkpoint['mIoU_for_epochs']


In [ ]:
net = bisenet_model.half().to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

best_mIoU = 0

for epoch in range(check_epoch,NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  current_step=0
  # Iterate over the dataset
  for images, labels in train_dataloader_B:
    # Bring data over the device of choice
    
    images = images.half().to(DEVICE)
    labels = labels.half().to(DEVICE)
   
    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    output1,output2,output3,output4,output5 = net(images)
    pred1 = output1.argmax(dim=1)
    pred2 = output2.argmax(dim=1)
    pred3 = output3.argmax(dim=1)
    pred4 = output4.argmax(dim=1)
    pred5 = output5.argmax(dim=1)
    
    loss1 = criterion(output1,labels.long())[labels!=255].mean()
    loss2 = criterion(output2,labels.long())[labels!=255].mean()
    loss3 = criterion(output3,labels.long())[labels!=255].mean()
    loss4 = criterion(output4,labels.long())[labels!=255].mean()
    loss5 = criterion(output5,labels.long())[labels!=255].mean()

    loss = loss1+loss2+loss3+loss4+loss5
    
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))
      
      mIoU = compute_mIoU(labels,pred1)
      print('Step {}, mIoU {}'.format(current_step, mIoU))
      
    loss.backward()

  
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  if epoch % 10 == 0:
    torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'loss_for_epochs' : loss_for_epochs,
            'mIoU_for_epochs' : mIoU_for_epochs,
            }, '/content/drive/MyDrive/magna/step2/long_esp_dataset_B_5e-3/'+ str(epoch) +'checkpoint.pt')

  loss_for_epochs.append(loss.item())
  mIoU_for_epochs.append(mIoU)
  # Step the scheduler
  scheduler.step() 

plotLoss(loss_for_epochs)
plotmIoU(mIoU_for_epochs)
print(loss_for_epochs)
print(mIoU_for_epochs)